In [1]:
# Standard improts
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np

# Setting width of jupyter NB
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### DATA Function

In [2]:
def getDBData(wellName = None, df_type = "perf"):
    '''
        df_type > "perf" as perforations, "prod" as production for wellName, wells, as all wells from database
    '''
    
    # SQLITE CONNECTION
    conn = sqlite3.connect(r'DB\PerforationProductionDB.sqlite')
    cursor  = conn.cursor()
    
    # PERFORATIONS QUERY
    if df_type == "perf" and wellName != None:
        SQL_PERF  = f'''SELECT WELLS.WELL, WELLS.ALIAS, PERFORATION.Open_Interval_Top, PERFORATION.Open_Interval_Bottom, PERFORATION.WO_date_Start, PERFORATION.WO_date_End, WELLS.Top_Of_Reservoir_m
              FROM WELLS INNER JOIN PERFORATION ON WELLS.ID = PERFORATION.fk_Well 
              WHERE (WELLS.WELL='{wellName}')'''
        df_perf = pd.read_sql(SQL_PERF, conn)
        
        df_perf["WO_date_Start"] = pd.to_datetime(df_perf["WO_date_Start"], format="%d/%m/%Y")
        df_perf["WO_date_End"] = pd.to_datetime(df_perf["WO_date_End"], format="%d/%m/%Y")
        
        df_perf.sort_values(["WO_date_End", "Open_Interval_Top"], inplace = True)
        
        conn.close()
        return df_perf
    # PRODUCTION QUERY
    elif df_type == "prod" and wellName != None:
        SQL_PROD = f'''SELECT PRODUCTION.DATE_LAST, PRODUCTION.OIL, PRODUCTION.WATER, PRODUCTION.GAS
              FROM PRODUCTION INNER JOIN WELLS ON WELLS.ID=PRODUCTION.fk_Well
              WHERE (WELLS.WELL='{wellName}')'''
        
        df_prod = pd.read_sql(SQL_PROD, conn)
        conn.close()

        df_prod["OIL"].fillna(0, inplace=True)
        df_prod["WATER"].fillna(0,inplace=True)
        df_prod["GAS"].fillna(0,inplace=True)
        df_prod["DATE_LAST"] = pd.to_datetime(df_prod["DATE_LAST"], format="%d/%m/%Y")
        df_prod.sort_values("DATE_LAST", inplace=True)
    #
        df_prod["OIL"] = df_prod["OIL"]/30
        df_prod["WATER"] = df_prod["WATER"]/30
        df_prod["GAS"] = df_prod["GAS"]/30
        return df_prod
    
    # WELL LIST QUERY
    elif df_type == "wells" and wellName == None:
        SQL_WELLS = "SELECT WELLS.WELL FROM WELLS"
        pd_wells = pd.read_sql(SQL_WELLS,conn)
        conn.close()
        return pd_wells["WELL"].unique()
    
    else:
        print('Dataframe type not dedfined as "perf" or "prod"')
        return 0     
    

## Charting Function

In [3]:
def plotChart(WellName, df_perf, df_prod):
    
    # 5 subplots with shared X axis
    fig, ax = plt.subplots(nrows=5, ncols=1, sharex='all')
    today =np.datetime64(datetime.today())

    # PLOTTING TOP OF RESERVOIR FOR WELL
    lineX=[df_perf["WO_date_Start"].min(),today]
    lineY=[df_perf["Top_Of_Reservoir_m"].unique(),df_perf["Top_Of_Reservoir_m"].unique()]

    # Dates of END_of_WO converted to list -> sorted
    listWOEndDates = df_perf["WO_date_End"].unique()
    lastListIndex = len(listWOEndDates) - 1


    # PLOT EACH PERFORATION FROM DATAFRAME.
    for a in df_perf.itertuples():
        minX = a[6]
    
        if np.where(listWOEndDates == minX)[0][0] == lastListIndex:
            maxX = today
        else:
            maxX = listWOEndDates[np.where(listWOEndDates == minX)[0]+1]
            maxX = maxX[0]
        
        ax[0].fill([minX,maxX,maxX,minX],[a[4],a[4],a[3],a[3]], label=(str(a[4])+" - "+str(a[3])))
        ax[1].fill([minX,maxX,maxX,minX],[a[4],a[4],a[3],a[3]], label=(str(a[4])+" - "+str(a[3])))
        ax[2].fill([minX,maxX,maxX,minX],[a[4],a[4],a[3],a[3]], label=(str(a[4])+" - "+str(a[3])))
    
        ax[0].plot([minX,maxX,maxX,minX, minX, minX],[a[4],a[4],a[3],a[3],a[3],a[4]],linewidth=0.95, linestyle='-', color='black')
        ax[1].plot([minX,maxX,maxX,minX, minX, minX],[a[4],a[4],a[3],a[3],a[3],a[4]],linewidth=0.95, linestyle='-', color='black')
        ax[2].plot([minX,maxX,maxX,minX, minX, minX],[a[4],a[4],a[3],a[3],a[3],a[4]],linewidth=0.95, linestyle='-', color='black')
    
    # TOP OF THE RESERVOIR LINE
    ax[0].plot(lineX, lineY, color="black", label="Top of Reservoir", linewidth=3, linestyle='--')
    ax[1].plot(lineX, lineY, color="black", label="Top of Reservoir", linewidth=3, linestyle='--')
    ax[2].plot(lineX, lineY, color="black", label="Top of Reservoir", linewidth=3, linestyle='--')
    
    #SECONDARY AXIS - WATER
    ax2_0 = ax[0].twinx()
    ax2_0.scatter(df_prod["DATE_LAST"], df_prod["WATER"], color = "blue", label="WATER SCM/DAY", alpha=0.5, edgecolors='black', linewidths=1, s=50)
    ax2_0.set_ylabel("WATER - Avg. 30 days SCM/day")
    
    #SECONDARY AXIS - OIL
    ax2_1 = ax[1].twinx()
    ax2_1.scatter(df_prod["DATE_LAST"], df_prod["OIL"], color = "green", label="OIL SCM/DAY", alpha=0.5, edgecolors='black', linewidths=1, s=50)
    ax2_1.set_ylabel("OIL - Avg. 30 days SCM/day")
    
    #SECONDARY AXIS - GAS
    ax2_2 = ax[2].twinx()
    ax2_2.scatter(df_prod["DATE_LAST"], df_prod["GAS"], color = "orange", label="GAS SCM/DAY", alpha=0.5, edgecolors='black', linewidths=1, s=50)
    ax2_2.set_ylabel("GAS - Avg. 30 days SCM/day")

    #WATERCUT
    ax[3].scatter(df_prod["DATE_LAST"], (df_prod["WATER"]/( df_prod["WATER"]+ df_prod["OIL"])),edgecolors='black', linewidths=1, s=50, color = "blue", label="WCUT SCM/SCM")

    #GOR
    ax[4].scatter(df_prod["DATE_LAST"], (df_prod["GAS"]/df_prod["OIL"]),edgecolors='black', linewidths=1, s=50, color = "red", label="GOR SCM/SCM")

    #LABLES ETC...
    ax[0].set_ylabel("Depth")
    ax[0].set_xlabel("Date")
    ax[0].set_title("PERFORATIONS VS TIME & WATER PRODUCTION"+ " - " + WellName )
    ax[0].invert_yaxis()

    ax[1].set_ylabel("Depth")
    ax[1].set_xlabel("Date")
    ax[1].set_title("OIL PRODUCTION"+ " - " + WellName )
    ax[1].legend()
    ax[1].invert_yaxis()

    ax[2].set_ylabel("Depth")
    ax[2].set_xlabel("Date")
    ax[2].set_title("GAS PRODUCTION" + " - " + WellName )
    ax[2].legend()
    ax[2].invert_yaxis()

    ax[3].set_ylabel("WATERCUT SCM/SCM")
    ax[3].set_xlabel("Date")
    ax[3].set_title("WATERCUT"+ " - " + WellName )
    ax[3].legend()

    ax[4].set_ylabel("GOR SCM/SCM")
    ax[4].set_xlabel("Date")
    ax[4].set_title("GOR"+ " - " + WellName )
    ax[4].legend()


    # LEGENDS TO THE RIGHT AND CENTER + TICKS
    #axCount = 0
    for a in ax:
        
        a.tick_params(axis='x', which='both', labelbottom=True)
        a.grid(visible=True, linestyle = '--', color='gray', linewidth=0.4)
        theBox = a.get_position()
        a.set_position([theBox.x0, theBox.y0, theBox.width*0.8, theBox.height])
        a.legend(loc='center left', bbox_to_anchor=(1.05,0.5))


        #axCount = axCount+1


    fig.set_figheight(15)
    fig.set_figwidth(25)

    plt.tight_layout()
    
    plt.show()
    

In [4]:
df_wells = getDBData(df_type="wells")

def doPloting(well):
        df_perf = getDBData(well, "perf")
        df_prod = getDBData(well, "prod")
        plotChart(well, df_perf, df_prod)
        
interact(doPloting, well = widgets.Dropdown(options = df_wells, description = "Select a well:"))



interactive(children=(Dropdown(description='Select a well:', options=('XY-3', 'XY-4', 'XY-5', 'XY-6AL', 'XY-7'…

<function __main__.doPloting(well)>